In [1]:
import pandas as pd
from sklearn.preprocessing import LabelEncoder, OneHotEncoder
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LinearRegression, ElasticNet
from sklearn.metrics import mean_squared_error, r2_score, mean_absolute_error
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import RandomForestRegressor, GradientBoostingRegressor
from sklearn.svm import SVR
from sklearn.neighbors import KNeighborsRegressor
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer
from xgboost import XGBRegressor

In [28]:
train = pd.read_csv("data/train_final.csv")
test = pd.read_csv("data/test_final.csv")
target = pd.read_csv("data/target.csv")

In [29]:
train.describe().T

,count,mean,std,min,25%,50%,75%,max
Rating,15730.0,4.012873,0.298440,0.0,3.9,4.0,4.20,5.0
actprice1,15730.0,1369.286777,1240.900227,42.0,699.0,999.0,1299.00,13499.0
norating1,15730.0,2925.868404,11605.447095,0.0,50.0,267.0,1379.25,289973.0
noreviews1,15730.0,408.397266,1737.269600,0.0,7.0,38.0,198.00,45448.0
star_5f,15730.0,1599.965035,6233.811271,0.0,30.0,153.0,789.00,151193.0
star_4f,15730.0,662.491545,2843.657075,0.0,12.0,61.0,304.00,74037.0
star_3f,15730.0,360.168023,1416.859022,0.0,7.0,34.0,172.00,34978.0
star_2f,15730.0,155.085188,558.650254,0.0,3.0,17.0,77.00,11705.0
star_1f,15730.0,275.500572,958.589075,0.0,6.0,30.0,140.00,18060.0
fulfilled1,15730.0,0.601526,0.489600,0.0,0.0,1.0,1.00,1.0


In [30]:
test.describe().T

,count,mean,std,min,25%,50%,75%,max
Rating,5244.0,4.015580,0.311346,0.0,3.9,4.0,4.20,5.0
actprice1,5244.0,1378.657895,1280.630070,139.0,699.0,999.0,1299.00,15999.0
norating1,5244.0,2988.580092,12881.253715,1.0,66.0,307.0,1428.00,289973.0
noreviews1,5244.0,415.491037,1910.726669,0.0,9.0,44.0,214.25,45448.0
star_5f,5244.0,1545.586003,6544.085444,0.0,32.0,156.0,756.50,151193.0
star_4f,5244.0,639.785469,2991.065223,0.0,12.0,61.0,310.25,74037.0
star_3f,5244.0,356.356789,1632.732834,0.0,7.0,34.0,166.00,49924.0
star_2f,5244.0,154.139969,611.006799,0.0,3.0,16.0,75.00,12629.0
star_1f,5244.0,260.807018,1017.789008,0.0,5.0,26.0,120.25,23139.0
fulfilled1,5244.0,0.604500,0.489004,0.0,0.0,1.0,1.00,1.0


In [31]:
train.dtypes

title          object
Rating        float64
maincateg      object
platform       object
actprice1       int64
norating1     float64
noreviews1    float64
star_5f       float64
star_4f       float64
star_3f       float64
star_2f         int64
star_1f       float64
fulfilled1      int64
combo            bool
category       object
dtype: object

In [32]:
train.columns

Index(['title', 'Rating', 'maincateg', 'platform', 'actprice1', 'norating1',
       'noreviews1', 'star_5f', 'star_4f', 'star_3f', 'star_2f', 'star_1f',
       'fulfilled1', 'combo', 'category'],
      dtype='object')

In [33]:
train.drop("title", axis=1, inplace=True)
# test_id = test.id
test.drop("title", axis=1, inplace=True)

X = train
y = target
assert X.shape[1] == test.shape[1], "X and test have different number of columns"

In [34]:
columns_to_one_hot = ["maincateg", "category", "platform", "combo"]
columns_to_standardize = ["actprice1", 'norating1', 'noreviews1', 'star_5f', 
    'star_4f', 'star_3f', 'star_2f', 'star_1f',]

In [35]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [16]:
# X_dev, X_test, y_dev, y_test = train_test_split(X_test, y_test, test_size=0.5, random_state=42)

In [36]:
len(X_train), len(X_test)

(12584, 3146)

In [37]:
label_encoder = LabelEncoder()
one_hot_encoder = OneHotEncoder(sparse=False)
standardizer = StandardScaler()

preprocessor = ColumnTransformer(
    [
        ("one_hot_encoder", one_hot_encoder, columns_to_one_hot),
        ("standardizer", standardizer, columns_to_standardize)],
    remainder="passthrough"
)

In [38]:
preprocessor.fit(X_train)
X_train = preprocessor.transform(X_train)
# X_dev = preprocessor.transform(X_dev)
X_test = preprocessor.transform(X_test)
test = preprocessor.transform(test)

In [39]:
assert X_train.shape[1] == test.shape[1], "X and test have different number of columns"

In [51]:
X_train.min(axis=0), test.min(axis=0)

(array([ 0.        ,  0.        ,  0.        ,  0.        ,  0.        ,
         0.        ,  0.        ,  0.        ,  0.        ,  0.        ,
         0.        ,  0.        ,  0.        ,  0.        ,  0.        ,
         0.        ,  0.        , -0.96847596, -0.25124309, -0.23788562,
        -0.258778  , -0.23556195, -0.25551057, -0.27874569, -0.28748864,
         0.        ,  0.        ]),
 array([ 0.        ,  0.        ,  0.        ,  0.        ,  0.        ,
         0.        ,  0.        ,  0.        ,  0.        ,  0.        ,
         0.        ,  0.        ,  0.        ,  0.        ,  0.        ,
         0.        ,  0.        , -0.99526439, -0.25115692, -0.23788562,
        -0.258778  , -0.23556195, -0.25551057, -0.27874569, -0.28748864,
         0.        ,  0.        ]))

# Models

## Base Model

In [45]:
lr = LinearRegression(positive=True)
lr.fit(X_train, y_train)
y_pred = lr.predict(X_test)
print("Linear Regression On Test Set:")
print("R2:", r2_score(y_test, y_pred))
print("MAE:", mean_absolute_error(y_test, y_pred))
print("RMSE:", mean_squared_error(y_test, y_pred)**0.5)
print("\n")
print("Linear Regression On Train Set:")
y_pred_t = lr.predict(X_train)
print("R2:", r2_score(y_train, y_pred_t))
print("MAE:", mean_absolute_error(y_train, y_pred_t))
print("RMSE:", mean_squared_error(y_train, y_pred_t)**0.5)

Linear Regression On Test Set:
R2: 0.8214036593164931
MAE: 181.5240980610299
RMSE: 287.34810108818095


Linear Regression On Train Set:
R2: 0.8023614612722689
MAE: 182.80386366119677
RMSE: 285.1834423376068


In [41]:
xgbr = XGBRegressor(n_estimators=1000, max_depth=4, learning_rate=0.1)
xgbr.fit(X_train, y_train)
y_pred = xgbr.predict(X_test)
print("XGBR On test Set:")
print("R2:", r2_score(y_test, y_pred))
print("MAE:", mean_absolute_error(y_test, y_pred))
print("RMSE:", mean_squared_error(y_test, y_pred)**0.5)
print("\n")
print("XGBR On Train Set:")
y_pred_t = xgbr.predict(X_train)
print("R2:", r2_score(y_train, y_pred_t))
print("MAE:", mean_absolute_error(y_train, y_pred_t))
print("RMSE:", mean_squared_error(y_train, y_pred_t)**0.5)

XGBR On test Set:
R2: 0.9085789875077412
MAE: 110.49827334142687
RMSE: 205.58672428778644


XGBR On Train Set:
R2: 0.9688170369830018
MAE: 75.62243969366192
RMSE: 113.27841915234771


In [63]:
rfr = RandomForestRegressor(n_estimators=1000, max_depth=6, n_jobs=4)
rfr.fit(X_train, y_train)
y_pred = rfr.predict(X_test)
print("rfr On Test Set:")
print("R2:", r2_score(y_test, y_pred))
print("MAE:", mean_absolute_error(y_test, y_pred))
print("RMSE:", mean_squared_error(y_test, y_pred)**0.5)
print("\n")
print("rfr On Train Set:")
y_pred_t = rfr.predict(X_train)
print("R2:", r2_score(y_train, y_pred_t))
print("MAE:", mean_absolute_error(y_train, y_pred_t))
print("RMSE:", mean_squared_error(y_train, y_pred_t)**0.5)

rfr On Test Set:
R2: 0.8640384138491943
MAE: 155.30687622438677
RMSE: 250.71491139561184


rfr On Train Set:
R2: 0.8624752515979218
MAE: 152.56851417410965
RMSE: 237.89149705847623


In [34]:
y_pred = rfr.predict(X_test)
print("rfr On Test Set:")
print("R2:", r2_score(y_test, y_pred))
print("MAE:", mean_absolute_error(y_test, y_pred))
print("RMSE:", mean_squared_error(y_test, y_pred)**0.5)
print("\n")
print("rfr On Train Set:")
y_pred_t = rfr.predict(X_train)
print("R2:", r2_score(y_train, y_pred_t))
print("MAE:", mean_absolute_error(y_train, y_pred_t))
print("RMSE:", mean_squared_error(y_train, y_pred_t)**0.5)

rfr On Test Set:
R2: 0.8843644285706092
MAE: 140.37400816642264
RMSE: 231.21594415304895


rfr On Train Set:
R2: 0.9038352083942
MAE: 130.08123969009722
RMSE: 198.92828440371628


In [30]:
svr = SVR()
svr.fit(X_train, y_train)
y_pred = svr.predict(X_test)
print("svr On Test Set:")
print("R2:", r2_score(y_test, y_pred))
print("MAE:", mean_absolute_error(y_test, y_pred))
print("MSE:", mean_squared_error(y_test, y_pred))
print("\n")
print("svr On Train Set:")
y_pred_t = svr.predict(X_train)
print("R2:", r2_score(y_train, y_pred_t))
print("MAE:", mean_absolute_error(y_train, y_pred_t))
print("MSE:", mean_squared_error(y_train, y_pred_t))

svr On Test Set:
R2: 0.41497833269179396
MAE: 257.3077333123637
MSE: 270468.10485040746


svr On Train Set:
R2: 0.4266329229337835
MAE: 244.90816648362159
MSE: 235944.43125092195


Let's do a grid search for the xgbr.

In [17]:
from sklearn.model_selection import GridSearchCV
params = {
    "n_estimators": [1500, 2000, 3000],
    "max_depth": [2, 4, 8],
    "learning_rate": [0.1,0.2, 0.3],
    "booster": ["gbtree"],
}

base_model = XGBRegressor()
grid_search = GridSearchCV(base_model, params, cv=3, n_jobs=-1, verbose=10)
grid_search.fit(X_train, y_train)

Fitting 3 folds for each of 27 candidates, totalling 81 fits
[CV 1/3; 1/27] START booster=gbtree, learning_rate=0.1, max_depth=2, n_estimators=1500[CV 3/3; 1/27] START booster=gbtree, learning_rate=0.1, max_depth=2, n_estimators=1500

[CV 2/3; 3/27] START booster=gbtree, learning_rate=0.1, max_depth=2, n_estimators=3000
[CV 3/3; 2/27] START booster=gbtree, learning_rate=0.1, max_depth=2, n_estimators=2000
[CV 2/3; 2/27] START booster=gbtree, learning_rate=0.1, max_depth=2, n_estimators=2000
[CV 1/3; 3/27] START booster=gbtree, learning_rate=0.1, max_depth=2, n_estimators=3000
[CV 2/3; 1/27] START booster=gbtree, learning_rate=0.1, max_depth=2, n_estimators=1500
[CV 1/3; 2/27] START booster=gbtree, learning_rate=0.1, max_depth=2, n_estimators=2000
[CV 2/3; 1/27] END booster=gbtree, learning_rate=0.1, max_depth=2, n_estimators=1500;, score=0.866 total time=  32.0s
[CV 3/3; 3/27] START booster=gbtree, learning_rate=0.1, max_depth=2, n_estimators=3000
[CV 3/3; 1/27] END booster=gbtree, lea

GridSearchCV(cv=3,
             estimator=XGBRegressor(base_score=None, booster=None,
                                    colsample_bylevel=None,
                                    colsample_bynode=None,
                                    colsample_bytree=None,
                                    enable_categorical=False, gamma=None,
                                    gpu_id=None, importance_type=None,
                                    interaction_constraints=None,
                                    learning_rate=None, max_delta_step=None,
                                    max_depth=None, min_child_weight=None,
                                    missing=nan, monotone_constraints=None,
                                    n_estimators=100, n_jobs=None,
                                    num_parallel_tree=None, predictor=None,
                                    random_state=None, reg_alpha=None,
                                    reg_lambda=None, scale_pos_weight=None,
       

In [18]:
grid_search.best_score_

0.8895123915370542

In [16]:
xgbr = XGBRegressor(learning_rate=0.3, max_depth=4, n_estimators=1000, booster="gbtree")
xgbr.fit(X_train, y_train)
y_pred = xgbr.predict(X_test)
print("XGBR On Test Set:")
print("R2:", r2_score(y_test, y_pred))
print("MAE:", mean_absolute_error(y_test, y_pred))
print("MSE:", mean_squared_error(y_test, y_pred))
print("\n")
print("XGBR On Train Set:")
y_pred_t = xgbr.predict(X_train)
print("R2:", r2_score(y_train, y_pred_t))
print("MAE:", mean_absolute_error(y_train, y_pred_t))
print("MSE:", mean_squared_error(y_train, y_pred_t))

XGBR On Test Set:
R2: 0.9094067675501246
MAE: 97.86719559986578
MSE: 41883.20067824357


XGBR On Train Set:
R2: 0.991287236873442
MAE: 37.40795654774622
MSE: 3585.360971610742


In [21]:
import numpy as np
y_pred = np.abs(xgbr.predict(X_test))
print("XGBR On Test Set:")
print("R2:", r2_score(y_test, y_pred))
print("MAE:", mean_absolute_error(y_test, y_pred))
print("MSE:", mean_squared_error(y_test, y_pred))
print("\n")
print("XGBR On Train Set:")
y_pred_t = np.abs(xgbr.predict(X_train))
print("R2:", r2_score(y_train, y_pred_t))
print("MAE:", mean_absolute_error(y_train, y_pred_t))
print("MSE:", mean_squared_error(y_train, y_pred_t))

XGBR On Test Set:
R2: 0.9094067675501246
MAE: 97.86719559986578
MSE: 41883.20067824357


XGBR On Train Set:
R2: 0.991287236873442
MAE: 37.40795654774622
MSE: 3585.360971610742


In [22]:
preds = xgbr.predict(test)

In [42]:
sample = pd.read_csv('data/Sample__submission.csv')

In [53]:
import numpy as np
preds = lr.predict(test)
sample['price1'] = np.abs(preds)
sample['price1'].describe()

count    5244.000000
mean      691.179944
std       596.309156
min        16.070312
25%       376.417969
50%       511.496094
75%       688.851562
max      7309.648438
Name: price1, dtype: float64

In [64]:
preds = rfr.predict(test)
sample['price1'] = np.abs(preds)
sample['price1'].describe()

count    5244.000000
mean      677.209561
std       536.053291
min       225.746408
25%       448.070001
50%       457.356946
75%       603.277634
max      3230.266872
Name: price1, dtype: float64

In [54]:
sample.to_csv('data/submission_3.csv', index=False)

In [28]:
X_train.shape

(12584, 27)